# Project Statement

The Film Junky Union, a new edgy community for classic movie enthusiasts, is developing a system for filtering and categorizing movie reviews. The goal is to train a model to automatically detect negative reviews. You'll be using a dataset of IMBD movie reviews with polarity labelling to build a model for classifying positive and negative reviews. It will need to have an F1 score of at least 0.85.

In [ ]:
import os

# Colab setup (only needed in Google Colab)
if os.path.exists("/content"):
    if not os.path.exists("Data_projects_TripleTen"):
        !git clone https://github.com/Olivenatural/Data_projects_TripleTen.git
    %cd Data_projects_TripleTen/Sprint14_Text_ML

In [ ]:
import pandas as pd

# ✅ GitHub / Colab sample file
DATA_PATH = "data/sample/imdb_reviews_sample.tsv"

# ✅ TripleTen full dataset (uncomment if running inside TripleTen)
# DATA_PATH = "/datasets/imdb_reviews.tsv"


df_reviews = pd.read_csv(DATA_PATH, sep="\t")

if "votes" in df_reviews.columns:
    df_reviews["votes"] = df_reviews["votes"].astype("Int64")

df_reviews.head()

In [ ]:
import os

repo_dir = "Data_projects_TripleTen"

if not os.path.exists(repo_dir):
    !git clone https://github.com/YOUR-USERNAME/Data_projects_TripleTen.git

%cd Data_projects_TripleTen/Sprint14_Text_ML

## Initialization

In [ ]:
import math

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from tqdm.auto import tqdm



In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
# the next line provides graphs of better quality on HiDPI screens
%config InlineBackend.figure_format = 'retina'

plt.style.use("seaborn-v0_8")

In [ ]:

# this is to use progress_apply, read more at https://pypi.org/project/tqdm/#pandas-integration
tqdm.pandas()


## Load Data

In [ ]:

# ✅ GitHub / Colab sample file
DATA_PATH = "data/sample/imdb_reviews_sample.tsv"

# ✅ TripleTen full dataset (uncomment if running inside TripleTen)
# DATA_PATH = "/datasets/imdb_reviews.tsv"


DATA_PATH = "data/sample/imdb_reviews_sample.tsv"
df_reviews = pd.read_csv(DATA_PATH, sep="\t")

if "votes" in df_reviews.columns:
    df_reviews["votes"] = df_reviews["votes"].astype("Int64")

df_reviews.head()

In [ ]:
print("Columns in df_reviews:", df_reviews.columns.tolist())

In [ ]:
SAMPLE_MODE = set(df_reviews.columns) == {"review", "pos", "ds_part"}
print("SAMPLE_MODE:", SAMPLE_MODE)

In [ ]:
if SAMPLE_MODE:
    print("Skipping this section in sample mode (columns not available).")
else:
    # your original code that uses extra columns
    dft1 = df_reviews[['tconst', 'start_year']].drop_duplicates()['start_year'].value_counts().sort_index()

## EDA

Let's check the number of movies and reviews over years.

In [ ]:
needed_cols = {"tconst", "start_year"}

if needed_cols.issubset(df_reviews.columns):
    dft1 = df_reviews[["tconst", "start_year"]].drop_duplicates()["start_year"] \
        .value_counts().sort_index()

    dft1 = dft1.reindex(index=np.arange(dft1.index.min(), max(dft1.index.max(), 2021))).fillna(0)

    ax = axs[0]  # keep your existing axes line if you already created axs
    ax.plot(dft1.index, dft1.values)
    ax.set_title("Reviews by start_year")
else:
    print("Skipping start_year plot — sample file does not include 'tconst' and 'start_year'.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 1, figsize=(16, 8))

needed_cols = {"tconst", "start_year", "pos"}

if needed_cols.issubset(df_reviews.columns):
    # --- Plot 1: Number of Movies Over Years ---
    ax = axs[0]
    dft1 = (df_reviews[["tconst", "start_year"]]
            .drop_duplicates()["start_year"]
            .value_counts()
            .sort_index())

    dft1 = dft1.reindex(index=np.arange(dft1.index.min(), max(dft1.index.max(), 2021))).fillna(0)
    dft1.plot(kind="bar", ax=ax)
    ax.set_title("Number of Movies Over Years")

    # --- Plot 2: Number of Reviews Over Years ---
    ax = axs[1]

    dft2 = df_reviews.groupby(["start_year", "pos"])["pos"].count().unstack()
    dft2 = dft2.reindex(index=np.arange(dft2.index.min(), max(dft2.index.max(), 2021))).fillna(0)
    dft2.plot(kind="bar", stacked=True, label="#reviews (neg, pos)", ax=ax)

    dft_total = df_reviews["start_year"].value_counts().sort_index()
    dft_total = dft_total.reindex(index=np.arange(dft_total.index.min(), max(dft_total.index.max(), 2021))).fillna(0)

    reviews_per_movie = (dft_total / dft1).fillna(0)
    axt = ax.twinx()
    reviews_per_movie.reset_index(drop=True).rolling(5).mean().plot(
        color="orange",
        label="reviews per movie (avg over 5 years)",
        ax=axt
    )

    lines, labels = axt.get_legend_handles_labels()
    ax.legend(lines, labels, loc="upper left")
    ax.set_title("Number of Reviews Over Years")

else:
    # Sample dataset only has review/pos/ds_part
    for ax in axs:
        ax.axis("off")
    axs[0].text(
        0.01, 0.6,
        "Skipped: Year-based plots require columns like 'tconst' and 'start_year'.\n"
        "Your sample file only includes: review, pos, ds_part.",
        fontsize=12
    )

fig.tight_layout()

### Number of Reviews Over Years — what I’m seeing

- **Volume is climbing over time.** The stacked bars get much taller in the later years, which means IMDb activity (reviews submitted) has grown a lot.
- **Positives vs. negatives rise together.** Both sentiments increase in roughly the same proportion each year. No big long-term tilt toward one class.
- **Reviews per movie are trending up.** The orange rolling line (avg. reviews per movie) moves upward, so newer titles tend to attract more reviews than older ones.
- **Edge effects at the end.** The very last year or two can be under-counted if the data isn’t complete—so I won’t over-interpret the final bars.

**So what (for modeling):**
- The dataset looks **roughly balanced** by sentiment across years → I probably don’t need heavy class-imbalance tricks.
- Language and topics may **drift over time** (new slang, streaming-era phrasing), so I’ll keep my split strict (fit vectorizer on train only) and watch for generalization on the test set.

<div class="alert alert-success">
  <b>Reviewer’s comment – Iteration 1:</b><br>
  Great job setting up the environment with all the required libraries and style configs. The EDA plots are clear, well-labeled, and your commentary ties the visual patterns directly to their modeling implications (class balance, drift, generalization).
</div>

Let's check the distribution of number of reviews per movie with the exact counting and KDE (just to learn how it may differ from the exact counting)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 5))

needed_cols = {"tconst", "review"}

if needed_cols.issubset(df_reviews.columns):
    ax = axs[0]
    dft = (df_reviews.groupby("tconst")["review"].count()
           .value_counts()
           .sort_index())
    dft.plot.bar(ax=ax)
    ax.set_title("Bar Plot of #Reviews Per Movie")

    ax = axs[1]
    dft = df_reviews.groupby("tconst")["review"].count()
    sns.kdeplot(dft, ax=ax)
    ax.set_title("KDE Plot of #Reviews Per Movie")

else:
    for ax in axs:
        ax.axis("off")
    axs[0].text(
        0.01, 0.6,
        "Skipped: This plot needs the 'tconst' column.\n"
        "Your sample file only includes: review, pos, ds_part.",
        fontsize=12
    )

fig.tight_layout()

### # Reviews per Movie — my notes

- I see **most movies have just a handful of reviews**. The bars are tallest at 1–3 and drop off fast.
- There’s a **long right tail** — a few titles get tons of reviews (blockbusters/cult favorites).
- The KDE curve backs this up: big peak at low counts, then a slow taper → **very skewed** (mean > median).

**What this means for my modeling**
- A small set of popular movies could **dominate the vocabulary**. I’m keeping the vectorizer fit on **train only** to avoid leakage.
- I’ll watch that results aren’t driven by those high-volume titles; if needed, I’ll sanity-check performance across different movies.
- For slower experiments (like BERT samples), I’ll **sample a diverse set of movies**, not just the popular ones.


In [ ]:
df_reviews['pos'].value_counts()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

needed_cols = {"ds_part", "rating"}

if needed_cols.issubset(df_reviews.columns):
    ax = axs[0]
    dft = df_reviews.query('ds_part == "train"')["rating"].value_counts().sort_index()
    dft = dft.reindex(index=np.arange(min(dft.index.min(), 1), max(dft.index.max(), 11))).fillna(0)
    dft.plot.bar(ax=ax)
    ax.set_ylim([0, 5000])
    ax.set_title("The train set: distribution of ratings")

    ax = axs[1]
    dft = df_reviews.query('ds_part == "test"')["rating"].value_counts().sort_index()
    dft = dft.reindex(index=np.arange(min(dft.index.min(), 1), max(dft.index.max(), 11))).fillna(0)
    dft.plot.bar(ax=ax)
    ax.set_ylim([0, 5000])
    ax.set_title("The test set: distribution of ratings")

else:
    for ax in axs:
        ax.axis("off")
    axs[0].text(
        0.01, 0.6,
        "Skipped: This plot needs the 'rating' column.\n"
        "Your sample file only includes: review, pos, ds_part.",
        fontsize=12
    )

fig.tight_layout()

### Ratings distribution (train vs. test) — my notes

- Both splits show the same shape: **lots of 1–2s and 8–10s**, and **fewer mid-range ratings (4–6)**. It’s that classic “U-shape” you see on IMDb.
- The train and test charts look **very similar in scale** (y-axis capped at 5k), so there’s **no obvious distribution shift** between splits.
- This suggests many reviews come from people with **strong opinions** (love it or hate it), while truly neutral takes are rarer.

**Implications for modeling**
- Extremes should be **easier** for the classifier; the **middle band** is where I expect more mistakes.
- I’ll keep an eye on **errors by rating bucket** to see if performance dips in the 4–6 range.

Distribution of negative and positive reviews over the years for two parts of the dataset

In [ ]:
fig, axs = plt.subplots(
    2, 2, figsize=(16, 8),
    gridspec_kw=dict(width_ratios=(2, 1), height_ratios=(1, 1))
)

needed_cols = {"ds_part", "pos", "start_year", "tconst"}

if needed_cols.issubset(df_reviews.columns):
    # --- Train: reviews per year (stacked) ---
    ax = axs[0][0]
    dft = (df_reviews.query('ds_part == "train"')
           .groupby(["start_year", "pos"])["pos"]
           .count()
           .unstack())
    dft.index = dft.index.astype("int")
    dft = dft.reindex(index=np.arange(dft.index.min(), max(dft.index.max(), 2020))).fillna(0)
    dft.plot(kind="bar", stacked=True, ax=ax)
    ax.set_title("The train set: #reviews of different polarities per year")

    # --- Train: distribution per movie (KDE) ---
    ax = axs[0][1]
    dft = (df_reviews.query('ds_part == "train"')
           .groupby(["tconst", "pos"])["pos"]
           .count()
           .unstack())

    if 0 in dft.columns:
        sns.kdeplot(dft[0], label="negative", ax=ax)
    if 1 in dft.columns:
        sns.kdeplot(dft[1], label="positive", ax=ax)

    ax.legend()
    ax.set_title("The train set: distribution of polarities per movie")

    # --- Test: reviews per year (stacked) ---
    ax = axs[1][0]
    dft = (df_reviews.query('ds_part == "test"')
           .groupby(["start_year", "pos"])["pos"]
           .count()
           .unstack())
    dft.index = dft.index.astype("int")
    dft = dft.reindex(index=np.arange(dft.index.min(), max(dft.index.max(), 2020))).fillna(0)
    dft.plot(kind="bar", stacked=True, ax=ax)
    ax.set_title("The test set: #reviews of different polarities per year")

    # --- Test: distribution per movie (KDE) ---
    ax = axs[1][1]
    dft = (df_reviews.query('ds_part == "test"')
           .groupby(["tconst", "pos"])["pos"]
           .count()
           .unstack())

    if 0 in dft.columns:
        sns.kdeplot(dft[0], label="negative", ax=ax)
    if 1 in dft.columns:
        sns.kdeplot(dft[1], label="positive", ax=ax)

    ax.legend()
    ax.set_title("The test set: distribution of polarities per movie")

else:
    # Sample dataset only has review/pos/ds_part → skip this 4-plot panel cleanly
    for row in axs:
        for ax in row:
            ax.axis("off")
    axs[0][0].text(
        0.01, 0.6,
        "Skipped: This 4-panel analysis needs 'start_year' and 'tconst'.\n"
        "Your sample file only includes: review, pos, ds_part.",
        fontsize=12
    )

fig.tight_layout()

### Sentiment over years (train & test) — my notes

- **Clear growth over time.** In both splits, the stacked bars get taller in later years, so the volume of reviews has increased a lot.
- **Positives and negatives track together.** Both classes rise at similar rates each year; I don’t see a big year-by-year skew toward one label.
- **Per-movie counts are skewed but similar by class.** The KDE curves (right panels) for positive vs. negative overlap heavily — most movies get few reviews, with a long tail of very popular titles.
- **Train vs. test look aligned.** The shapes of both the yearly bars and the KDE curves are very similar between the splits, which suggests **no obvious distribution shift**.

**Implications for the model**
- Class balance looks stable over time; I don’t need heavy rebalancing tricks.
- Because review volume changes by year, I’ll continue to **fit vectorizers on train only** and watch for any time-based drift in test performance.


<div class="alert alert-success">
  <b>Reviewer’s comment – Iteration 1:</b><br>
  Excellent EDA continuation — the plots on reviews per movie, rating distributions, and sentiment over time are well-structured and accompanied by thoughtful commentary. You’ve clearly explained both the patterns (right-skewed distributions, U-shaped ratings, class balance over time) and their implications for modeling, which is exactly what we’re looking for.
</div>

## Evaluation Procedure

Composing an evaluation routine which can be used for all models in this project

In [ ]:

import sklearn.metrics as metrics

def evaluate_model(model, train_features, train_target, test_features, test_target):

    eval_stats = {}

    fig, axs = plt.subplots(1, 3, figsize=(20, 6))

    for type, features, target in (('train', train_features, train_target), ('test', test_features, test_target)):

        eval_stats[type] = {}

        pred_target = model.predict(features)
        pred_proba = model.predict_proba(features)[:, 1]

        # F1
        f1_thresholds = np.arange(0, 1.01, 0.05)
        f1_scores = [metrics.f1_score(target, pred_proba>=threshold) for threshold in f1_thresholds]

        # ROC
        fpr, tpr, roc_thresholds = metrics.roc_curve(target, pred_proba)
        roc_auc = metrics.roc_auc_score(target, pred_proba)
        eval_stats[type]['ROC AUC'] = roc_auc

        # PRC
        precision, recall, pr_thresholds = metrics.precision_recall_curve(target, pred_proba)
        aps = metrics.average_precision_score(target, pred_proba)
        eval_stats[type]['APS'] = aps

        if type == 'train':
            color = 'blue'
        else:
            color = 'green'

        # F1 Score
        ax = axs[0]
        max_f1_score_idx = np.argmax(f1_scores)
        ax.plot(f1_thresholds, f1_scores, color=color, label=f'{type}, max={f1_scores[max_f1_score_idx]:.2f} @ {f1_thresholds[max_f1_score_idx]:.2f}')
        # setting crosses for some thresholds
        for threshold in (0.2, 0.4, 0.5, 0.6, 0.8):
            closest_value_idx = np.argmin(np.abs(f1_thresholds-threshold))
            marker_color = 'orange' if threshold != 0.5 else 'red'
            ax.plot(f1_thresholds[closest_value_idx], f1_scores[closest_value_idx], color=marker_color, marker='X', markersize=7)
        ax.set_xlim([-0.02, 1.02])
        ax.set_ylim([-0.02, 1.02])
        ax.set_xlabel('threshold')
        ax.set_ylabel('F1')
        ax.legend(loc='lower center')
        ax.set_title(f'F1 Score')

        # ROC
        ax = axs[1]
        ax.plot(fpr, tpr, color=color, label=f'{type}, ROC AUC={roc_auc:.2f}')
        # setting crosses for some thresholds
        for threshold in (0.2, 0.4, 0.5, 0.6, 0.8):
            closest_value_idx = np.argmin(np.abs(roc_thresholds-threshold))
            marker_color = 'orange' if threshold != 0.5 else 'red'
            ax.plot(fpr[closest_value_idx], tpr[closest_value_idx], color=marker_color, marker='X', markersize=7)
        ax.plot([0, 1], [0, 1], color='grey', linestyle='--')
        ax.set_xlim([-0.02, 1.02])
        ax.set_ylim([-0.02, 1.02])
        ax.set_xlabel('FPR')
        ax.set_ylabel('TPR')
        ax.legend(loc='lower center')
        ax.set_title(f'ROC Curve')

        # PRC
        ax = axs[2]
        ax.plot(recall, precision, color=color, label=f'{type}, AP={aps:.2f}')
        # setting crosses for some thresholds
        for threshold in (0.2, 0.4, 0.5, 0.6, 0.8):
            closest_value_idx = np.argmin(np.abs(pr_thresholds-threshold))
            marker_color = 'orange' if threshold != 0.5 else 'red'
            ax.plot(recall[closest_value_idx], precision[closest_value_idx], color=marker_color, marker='X', markersize=7)
        ax.set_xlim([-0.02, 1.02])
        ax.set_ylim([-0.02, 1.02])
        ax.set_xlabel('recall')
        ax.set_ylabel('precision')
        ax.legend(loc='lower center')
        ax.set_title(f'PRC')

        eval_stats[type]['Accuracy'] = metrics.accuracy_score(target, pred_target)
        eval_stats[type]['F1'] = metrics.f1_score(target, pred_target)

    df_eval_stats = pd.DataFrame(eval_stats)
    df_eval_stats = df_eval_stats.round(2)
    df_eval_stats = df_eval_stats.reindex(index=('Accuracy', 'F1', 'APS', 'ROC AUC'))

    print(df_eval_stats)

    return


## Normalization

We assume all models below accepts texts in lowercase and without any digits, punctuations marks etc.

In [ ]:
df_reviews['review_norm'] = (df_reviews['review']
                            .str.lower()
                            .str.replace(r'<br\s*/?>', ' ', regex=True)
                            .str.replace(r'<[^>]+>', ' ', regex=True)
                            .str.replace(r"[^a-z\s']", ' ', regex=True)
                            .str.replace(r'\s+', ' ', regex=True)
                            .str.strip()
                            )


<div class="alert alert-success">
  <b>Reviewer’s comment – Iteration 1:</b><br>
  Great start on text preprocessing! You’re normalizing reviews by lowercasing, removing HTML tags, non-alphabetic characters, and trimming whitespace — all important steps to clean noisy raw text. The chained `.str.replace(..., regex=True)` is clear and concise.
</div>

Next, I'm going to tell TF-IDF to keep apostrophies in tokens. Sklearn's default token pattern splits "don't" into "don" and "t". I can keep it as one token by setting token_pattern:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    ngram_range=(1,2),
    min_df=3, max_df=0.9,
    sublinear_tf=True,
    strip_accents='unicode',
    token_pattern=r"(?u)\b[a-z][a-z']+\b"
)

<div class="alert alert-success">
  <b>Reviewer’s comment – Iteration 1:</b><br>
  Excellent adjustment on the TF-IDF vectorizer! By customizing `token_pattern` to keep apostrophes, you ensure contractions like “don’t” or “can’t” stay intact, which preserves sentiment-carrying tokens. Good use of other parameters too (`ngram_range`, `min_df`, `max_df`, `sublinear_tf`, `strip_accents`), showing awareness of both signal strength and noise control.
</div>

## Train / Test Split

Luckily, the whole dataset is already divided into train/test one parts. The corresponding flag is 'ds_part'.

In [ ]:

df_reviews_train = df_reviews.query('ds_part == "train"').copy()
df_reviews_test = df_reviews.query('ds_part == "test"').copy()

train_target = df_reviews_train['pos']
test_target = df_reviews_test['pos']

print(df_reviews_train.shape)
print(df_reviews_test.shape)


## Working with models

### Model 0 - Constant Baseline

A trivial classifier that predicts the same class every time. I'll set contant=0(always "negative"). This should produce F1=0 for the positive class and gives me a foundation to beat.

In [ ]:
from sklearn.dummy import DummyClassifier

In [ ]:
# 1) Text features
features_train_text = df_reviews_train['review_norm']
features_test_text = df_reviews_test['review_norm']
target_train = train_target.astype(int)
target_test = test_target.astype(int)

In [ ]:
# 2) Vectorize
tfidf_const = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=3, max_df=0.9,
    sublinear_tf=True,
    strip_accents='unicode'
)
features_train_const = tfidf_const.fit_transform(features_train_text)
features_test_const = tfidf_const.transform(features_test_text)

# 3) Dummy Classifier: "constant"
dummy_constant = DummyClassifier(strategy='constant', constant=0, random_state=42)
dummy_constant.fit(features_train_const, target_train)

In [ ]:
# 4) Evaluation
evaluate_model(dummy_constant, features_train_const, target_train, features_test_const, target_test);

<div class="alert alert-success">
  <b>Reviewer’s comment – Iteration 1:</b><br>
  Great work establishing a baseline with `DummyClassifier` and immediately evaluating it using your `evaluate_model()` function. This ensures you have a clear benchmark to compare future models against. Nice job also keeping the vectorizer fit on train only, then transforming test separately — that prevents data leakage.
</div>

### Model 0 — Constant Baseline (always predicts the same label)

**Results (train / test):**  
- **Accuracy:** 0.50 / 0.50  
- **F1:** 0.00 / 0.00  
- **ROC AUC:** 0.50 / 0.50  
- **Average Precision (APS):** 0.50 / 0.50

**What this means:**  
- The data is roughly balanced, so always guessing one label lands at ~50% accuracy by chance.  
- Because the model never predicts the positive class, recall for positives is 0 → **F1 is 0**.  
- **ROC AUC** and **APS** at 0.50 indicate no real skill—basically coin-flip performance.

**Takeaway & next step:**  
This baseline just proves the pipeline runs. Now I'll switch to real features (**TF-IDF with unigrams + bigrams**) and a proper classifier (**Logistic Regression**) to push **F1 ≥ 0.85**.


<div class="alert alert-success">
  <b>Reviewer’s comment – Iteration 1:</b><br>
  Excellent work defining a clear **baseline model** with `DummyClassifier`, evaluating it with multiple metrics, and writing a concise conclusion. The commentary connects the numbers (accuracy, F1, ROC AUC, APS) to their meaning, and you’ve explained why the baseline performs as it does. The clear “takeaway & next step” transition sets up the move to real models perfectly.
</div>

### Model 1 - NLTK, TF-IDF and LR


Plan: tokenize → preserve negation (turn “n’t” → **“not”**) → remove most stopwords (but **keep** not/no/nor) → lemmatize → TF-IDF (uni+bi-grams) → Logistic Regression. I’ll fit the vectorizer on **train only**, transform **test**, and evaluate with the same `evaluate_model()` helper.

In [ ]:
import nltk, re, html
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
# 0) Natural Language resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# 1) review_norm, Light/Safe normalization
if 'review_norm' not in df_reviews.columns:
    df_reviews['review_norm'] = (df_reviews['review']
        .str.lower()
        .replace(r'<br\s*/?>', ' ', regex=True)
        .str.replace(r'<[^>]+>', ' ', regex=True)
        .str.replace(r"[^a-z\s']", ' ', regex=True)
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip()
    )

# 2) Train/Test splits recreation
df_reviews_train = df_reviews.query('ds_part == "train"').copy()
df_reviews_test = df_reviews.query('ds_part == "test"').copy()
train_target = df_reviews_train['pos'].astype(int)
test_target = df_reviews_test['pos'].astype(int)

# 3) NLTK preprocessing(negation)
negation_keep = {"not", "no", "nor", "n't"}
stop_words = set(stopwords.words('english')) - negation_keep
lemmatizer = WordNetLemmatizer()

def text_preprocessing_1(text:str) -> str:
    t = str(text)
    toks = nltk.word_tokenize(t)
    cleaned = []
    for tok in toks:
        tok = tok.lower()
        if tok == "n't":
            tok = "not"
        if not re.search(r"[a-z]", tok):
            continue
        if tok in stop_words:
            continue
        tok = lemmatizer.lemmatize(tok)
        cleaned.append(tok)
    return " ".join(cleaned)



In [ ]:
import nltk

# 4) 'review_nltk' columns

for p in ["punkt", "stopwords", "wordnet", "omw-1.4", "punkt_tab"]:
    nltk.download(p)

if "review_norm" in df_reviews_train.columns and "review_norm" in df_reviews_test.columns:
    df_reviews_train["review_nltk"] = df_reviews_train["review_norm"].progress_apply(text_preprocessing_1)
    df_reviews_test["review_nltk"] = df_reviews_test["review_norm"].progress_apply(text_preprocessing_1)
else:
    print("Skipping: 'review_norm' not found in train/test dataframes.")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# 5) TF-IDF Vectorization for Model 1
# ✅ Make sure targets exist (pos = 1 positive, 0 negative)
train_target = df_reviews_train["pos"]
test_target = df_reviews_test["pos"]

# ✅ TF-IDF Vectorization (sample-friendly)
# If your sample is small, min_df=3 can cause "empty vocabulary" or missing terms.
min_df_value = 1 if len(df_reviews_train) < 50 else 3

tfidf_vectorizer_1 = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=min_df_value,
    max_df=0.9,
    sublinear_tf=True,
    strip_accents="unicode"
)

train_features_1 = tfidf_vectorizer_1.fit_transform(df_reviews_train["review_nltk"])
test_features_1 = tfidf_vectorizer_1.transform(df_reviews_test["review_nltk"])

# ✅ Logistic Regression Model
model_1 = LogisticRegression(random_state=42, max_iter=1000)
model_1.fit(train_features_1, train_target)


In [ ]:
evaluate_model(model_1, train_features_1, train_target, test_features_1, test_target)

<div class="alert alert-success">
  <b>Reviewer’s comment – Iteration 1:</b><br>
  Excellent work! You handled negations smartly (`n't` → `not`), preserved key stopwords, applied lemmatization, and clearly documented your preprocessing function. Training TF-IDF on train only and transforming test shows correct awareness of data leakage. Logistic Regression is a strong baseline for text classification, and your evaluation integrates seamlessly with `evaluate_model()`.
</div>


### Model 3 — spaCy + TF-IDF + Logistic Regression

**Results (train / test):**  
- **Accuracy:** 0.95 / 0.89  
- **F1:** 0.95 / 0.89  
- **Average Precision (APS):** 0.99 / 0.96  
- **ROC AUC:** 0.99 / 0.96  

**summary:**  
- This model clearly **meets the goal** (test **F1 = 0.89 ≥ 0.85**).  
- Very high train scores and a modest drop on test suggest a **good fit with mild overfitting**, which is normal for text models.  
- Strong **ROC AUC** and **APS** on test show it ranks positive reviews very well and maintains solid precision-recall balance.

**Note:**  
- Performance is **very close** to the NLTK version; if I need the last bit of F1, I can try small tweaks (e.g., tuning `C`, adjusting `min_df`, or experimenting with stopword handling), but this is already a strong result.


### Model 3 - spaCy, TF-IDF and LR

I’ll lemmatize text with spaCy, keep negations (e.g., “not”), vectorize with TF-IDF (uni+bi-grams), and train Logistic Regression. The vectorizer is fit on **train only** and evaluated on **test** with `evaluate_model()`.

In [ ]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# 1) spaCy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [ ]:
# 2) spaCy preprocessing (light/clean text)
def text_preprocessing_3(text: str) -> str:

    doc = nlp(str(text))
    lemmas = [tok.lemma_.lower() for tok in doc if not tok.is_space]
    return " ".join(lemmas)


In [ ]:
# 3) Preprocessed columns
df_reviews_train["review_spacy"] = df_reviews_train["review_norm"].progress_apply(text_preprocessing_3)
df_reviews_test["review_spacy"] = df_reviews_test["review_norm"].progress_apply(text_preprocessing_3)

# 4) Vectorized(Train/Text)
tfidf_3 = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.9,
    sublinear_tf=True,
    strip_accents="unicode"
)
train_features_3 = tfidf_3.fit_transform(df_reviews_train["review_spacy"])
test_features_3 = tfidf_3.transform(df_reviews_test["review_spacy"])



In [ ]:
# 5) Logistic Regression
model_3 = LogisticRegression(
    C=4.0,
    solver="liblinear",
    max_iter=2000,
    random_state=42
)
model_3.fit(train_features_3, train_target.astype(int))

In [ ]:
# 6) Evaluation
evaluate_model(
    model_3,
    train_features_3, train_target.astype(int),
    test_features_3, test_target.astype(int)
)

<div class="alert alert-success">
  <b>Reviewer’s comment – Iteration 1:</b><br>
  Excellent job building the spaCy preprocessing pipeline and integrating it with TF-IDF and Logistic Regression. The evaluation shows a strong model with test F1 = 0.89 (above the 0.85 target), and you clearly documented your methodology, metrics, and interpretation. The plots (F1 vs threshold, ROC, PRC) add valuable depth to your analysis and support your conclusions.
</div>

<div class="alert alert-warning">
  <b>Reviewer’s comment – Iteration 1:</b><br>
  Suggestion: the results show very high training performance (F1 ≈ 0.95, ROC AUC = 0.99) compared to slightly lower test performance (F1 ≈ 0.89, ROC AUC = 0.96). This indicates mild <b>overfitting</b>, which is common in text models. You might mention or test regularization adjustments (e.g., lowering `C`) or slight changes to `min_df` to see if they reduce the gap. Not critical here, since the model already meets the project goal.
</div>


### Model 4 - spaCy, TF-IDF and LGBMClassifier

I’m keeping the same text representation as Model 3 (spaCy lemmas → TF-IDF with uni+bi-grams), but switching the classifier to **LightGBM**. Gradient boosting can sometimes grab non-linear interactions that linear models miss.

**Plan**
- Use the `review_spacy` text (lemmatized with spaCy).
- Vectorize with TF-IDF (fit on train only; transform test).
- Train `LGBMClassifier` and evaluate with the same helper (`evaluate_model`), which uses `predict_proba`.

**Notes**
- LightGBM handles sparse TF-IDF directly.
- If LightGBM isn’t installed locally, I’ll see an import error and can install it with: `pip install lightgbm`.

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# 1) TF-IDF on spaCy lemmas (TRAIN)
tfidf_4 = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.9,
    sublinear_tf=True,
    strip_accents='unicode'
)
train_features_4 = tfidf_4.fit_transform(df_reviews_train['review_spacy'])
test_features_4  = tfidf_4.transform(df_reviews_test['review_spacy'])

# 2) LightGBM
model_4 = LGBMClassifier(
    n_estimators=300,
    learning_rate=0.1,
    num_leaves=31,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42,
    n_jobs=-1
)

In [ ]:
# 4) Trained
model_4.fit(train_features_4, train_target.astype(int))

In [ ]:
# 5) Evaluation
evaluate_model(
    model_4,
    train_features_4, train_target.astype(int),
    test_features_4, test_target.astype(int)
)

### Model 4 — spaCy + TF-IDF + LightGBM

**Results (train / test):**  
- **Accuracy:** 0.99 / 0.89  
- **F1:** 0.99 / 0.89  
- **Average Precision (APS):** 1.00 / 0.95  
- **ROC AUC:** 1.00 / 0.96  

**Plain-English takeaways:**  
- The model **meets the target** with **F1 = 0.89** on the test set.  
- Train scores are near-perfect while test is lower → **mild overfitting**, which is common on high-dimensional sparse text.  
- Strong **ROC AUC** and **APS** show good ranking and precision–recall behavior.

**Notes vs. Logistic Regression:**  
- Performance is **very close** to the LR model; this is typical—linear models often excel on TF-IDF features.  
- LightGBM didn’t beat LR here, but it’s still a solid alternative.

**Possible tweaks:**  
- Tune `num_leaves`, `n_estimators`, and `learning_rate`.  
- Try dimensionality reduction (e.g., **TruncatedSVD** to 300–500 comps) before LightGBM to reduce sparsity.  
- Consider threshold tuning to squeeze a bit more test F1.


<div class="alert alert-success">
  <b>Reviewer’s comment – Iteration 1:</b><br>
  Excellent work with your LightGBM experiment! You correctly reused the spaCy + TF-IDF pipeline, applied solid hyperparameters, and clearly documented results in a markdown conclusion. The test F1 = 0.89 meets the project target, and your plain-English takeaways (balanced with notes on overfitting and LR comparison) make the results easy to understand. Highlighting possible tweaks (regularization, dimensionality reduction, threshold tuning) shows good forward thinking.
</div>

###  Model 9 - BERT

I’ll convert texts into **BERT [CLS] embeddings** using `bert-base-uncased`, then train a simple **Logistic Regression** on those embeddings. Because full IMDB on CPU can take hours, I’m running a **small sample (≈200 train / 200 test)** to compare against the TF-IDF models. If a GPU is available, the code will use it automatically; otherwise it runs on CPU.

In [ ]:
import torch
import transformers
import math
import numpy as np
from tqdm.auto import tqdm

In [ ]:

tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
config = transformers.BertConfig.from_pretrained('bert-base-uncased')
model = transformers.BertModel.from_pretrained('bert-base-uncased')


In [ ]:

def BERT_text_to_embeddings(texts, max_length=512, batch_size=100, force_device=None, disable_progress_bar=False):

    ids_list = []
    attention_mask_list = []

    # 1) Tokenization
    enc = tokenizer(
        list(texts),
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True
    )
    ids_list = enc['input_ids']
    attention_mask_list = enc['attention_mask']

    # 2) Device
    if force_device is not None:
        device = torch.device(force_device)
    else:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model.to(device)
    if not disable_progress_bar:
        print(f'Using the {device} device.')

    # 3) gettings embeddings in batches
    embeddings = []
    n_batches = math.ceil(len(ids_list) / batch_size)

    for i in tqdm(range(math.ceil(len(ids_list)/batch_size)), disable=disable_progress_bar):
        start = batch_size * i
        end = batch_size * (i + 1)

        ids_batch = torch.LongTensor(ids_list[batch_size*i:batch_size*(i+1)]).to(device)
        attention_mask_batch = torch.LongTensor(attention_mask_list[start:end]).to(device)

        with torch.no_grad():
            model.eval()
            outputs = model(input_ids=ids_batch, attention_mask=attention_mask_batch)
            cls_batch = outputs.last_hidden_state[:, 0, :]
            embeddings.append(cls_batch.detach().cpu().numpy())

    return np.concatenate(embeddings, axis=0)


In [ ]:
# 1) Sample ~ 200 from each split
n_train = min(200, len(df_reviews_train))
n_test = min(200, len(df_reviews_test))

train_sample = df_reviews_train.sample(n=n_train, random_state=42)
test_sample = df_reviews_test.sample(n=n_test, random_state=42)

features_train_bert = train_sample['review_norm']
target_train_bert = train_sample['pos'].astype(int).values

features_test_bert = test_sample['review_norm']
target_test_bert = test_sample['pos'].astype(int).values

# 2) Embeddings
train_features_9 = BERT_text_to_embeddings(features_train_bert, max_length=256, batch_size=32)
test_features_9 = BERT_text_to_embeddings(features_test_bert, max_length=256, batch_size=32)

In [ ]:
print("Embeddings shape (train/test):", train_features_9.shape, test_features_9.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
model_9 = LogisticRegression(
    C=2.0, max_iter=2000, solver='lbfgs', random_state=42
)

model_9.fit(train_features_9, target_train_bert)

evaluate_model(
    model_9,
    train_features_9, target_train_bert,
    test_features_9, target_test_bert
)

<div class="alert alert-success">
  <b>Reviewer’s comment – Iteration 1:</b><br>
  Great effort experimenting with BERT! You correctly sampled a manageable subset, generated embeddings, and trained a classifier on top. Your batching logic, device management, and use of `with torch.no_grad()` are all implemented properly. It shows you understand both the heavy computational cost of transformers and how to integrate them into your pipeline.
</div>

<div class="alert alert-warning">
  <b>Reviewer’s comment – Iteration 1:</b><br>
  Suggestion: since BERT is computationally demanding and TripleTen doesn’t support GPU acceleration, it’s expected that running this code may crash or kill the kernel. For the project submission, it’s completely fine to note that you attempted this experiment but could not fully evaluate results due to hardware limits. Including just the setup and rationale (as you’ve done) is enough to demonstrate understanding.
</div>


In [ ]:
# Embeddings saved to disk
import numpy as np

np.savez_compressed(
    'features_9.npz',
    train_features_9=train_features_9,
    test_features_9=test_features_9
)
print("saved to features_9.npz")

In [ ]:
# Saved indices of the sampled rows
np.savez_compressed(
    'features_9_meta.npz',
    train_index=getattr(train_sample.index, "values", None),
    test_index=getattr(test_sample.index, "values", None)
)

train_features_9.shape[0] == len(target_train_bert)
test_features_9.shape[0] == len(target_test_bert)

### Model X — Results

**Results (train / test):**  
- **Accuracy:** 1.00 / 0.76  
- **F1:** 1.00 / 0.74  
- **Average Precision (APS):** 1.00 / 0.83  
- **ROC AUC:** 1.00 / 0.85

**What this tells me:**  
- The model is **perfect on train** but drops notably on test → classic **overfitting** (or potential leakage).  
- Despite decent ranking ability on test (ROC AUC **0.85**, APS **0.83**), the **operating F1 = 0.74** is **below the project target (≥ 0.85)** and below my stronger baselines.

**Checks I should make right away:**  
- Ensure the vectorizer/feature extractor was **fit on train only** and then **transformed** test (no leakage).  
- Confirm train/test separation for any learned preprocessing (e.g., SVD fit, BERT embeddings, scalers).  
- Verify I’m not accidentally mixing columns/splits when building features.

**How I’ll try to fix it:**  
- **Regularize more:**  
  - Logistic Regression → lower `C` (e.g., 1.0 → 0.5 or 0.25).  
  - LightGBM → reduce `num_leaves`, increase `min_child_samples`, use smaller `feature_fraction`.  
- **Dimensionality reduction:** Try TF-IDF → **TruncatedSVD (300–500)** → classifier to reduce sparsity.  
- **Threshold tuning:** Pick the F1-optimal threshold from my F1-vs-threshold curve instead of 0.50.  
- **Calibrate probabilities:** `CalibratedClassifierCV` (for SVMs) or `class_weight='balanced'` if needed.  
- **Data-side tweaks:** Preserve negations (“n’t” → “not”), keep bigrams, and add a few hard negative/irony examples.

**Bottom line:** As-is, this model **doesn’t meet the target** on test. I’ll either improve it with the steps above or proceed with my better-performing model (e.g., the spaCy + TF-IDF + LR that hit ~**0.89–0.90 F1** on test) as the primary solution.


## My Reviews

### New hand-labeled reviews for out-of-sample testing

Below are fresh reviews I wrote with **known ground truth** so I can test my models on text they’ve never seen.  
- `gold = 1` → positive  
- `gold = 0` → negative  

I included tricky cases (negation, contrast, mild sarcasm, mixed sentiment) to stress-test the classifiers.

In [ ]:
import pandas as pd

my_reviews = pd.DataFrame([
    {"review": "I didn't expect much, but this was a delightful surprise.", "gold": 1},
    {"review": "Not even the soundtrack could save this mess.", "gold": 0},
    {"review": "The pacing isn't perfect, yet I left the theater smiling.", "gold": 1},
    {"review": "I would not recommend this to my worst enemy.", "gold": 0},
    {"review": "Smart, heartfelt, and beautifully shot.", "gold": 1},
    {"review": "What a waste of two hours.", "gold": 0},
    {"review": "It's not bad at all—actually pretty great.", "gold": 1},
    {"review": "Jokes land flat, and the lead seems lost.", "gold": 0},
    {"review": "I cried, I laughed, I want to see it again.", "gold": 1},
    {"review": "The plot twists are obvious and tiresome.", "gold": 0},
    {"review": "Better than the last one by a mile.", "gold": 1},
    {"review": "I tried twice and still couldn't finish it.", "gold": 0},
    {"review": "A cozy, feel-good story I'll rewatch.", "gold": 1},
    {"review": "Meh—forgettable and bland.", "gold": 0},
    {"review": "Uneven first half, strong finish; overall I liked it.", "gold": 1},
    {"review": "Technically impressive, emotionally empty.", "gold": 0},
    {"review": "Not only entertaining but also thoughtful.", "gold": 1},
    {"review": "I kept checking the time—never a good sign.", "gold": 0},
    {"review": "Yeah, totally 'hilarious'—I didn't laugh once.", "gold": 0},  # sarcasm
    {"review": "This is the kind of movie that stays with you.", "gold": 1},
    {"review": "I regret renting this.", "gold": 0},
], columns=["review", "gold"])

# THE SAME normalization I used for the main dataset
my_reviews["review_norm"] = (
    my_reviews["review"]
      .str.lower()
      .str.replace(r"<br\s*/?>", " ", regex=True)
      .str.replace(r"<[^>]+>", " ", regex=True)
      .str.replace(r"[^a-z\s']", " ", regex=True)  # keep apostrophes
      .str.replace(r"\s+", " ", regex=True)
      .str.strip()
)

def print_preds(name, probs, texts, gold, threshold=0.5, preview_chars=100):
    preds = (probs >= threshold).astype(int)
    acc = (preds == gold).mean()
    print(f"\n{name} - mini-set accuracy @ {threshold:.2f}: {acc:.2%}")
    for i, review in enumerate(texts.str.slice(0, preview_chars)):
        print(f"{probs[i]:.2f}:  {review}")

<div class="alert alert-success">
  <b>Reviewer’s comment – Iteration 1:</b><br>
  Great initiative creating your own small labeled dataset for validation! Normalizing these reviews with the same pipeline ensures consistency, and the `print_preds` helper is a neat way to quickly inspect model probabilities, gold labels, and predictions. This shows strong practical awareness of testing models on unseen, real-like data.
</div>

### Model 1

### Model 1 — Predictions on my hand-labeled reviews

I transform the normalized texts with the same TF-IDF used to train Model 1, get the predicted **probability of positive**, print the first 100 chars of each review with the **predicted label**, and—if `gold` is present—compute quick accuracy/F1 on this mini set.


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import numpy as np

texts = my_reviews['review_norm']

X_m1 = tfidf_vectorizer_1.transform(texts)
my_reviews_pred_prob = model_1.predict_proba(X_m1)[:, 1]

threshold = 0.45
pred_labels = (my_reviews_pred_prob >= threshold).astype(int)


print("Model 1 — per-review predictions (p=probability of positive, pred=label)")
for i, preview in enumerate(my_reviews['review'].str.slice(0, 100)):
    print(f"p={my_reviews_pred_prob[i]:.2f}  pred={pred_labels[i]}  |  {preview}")

# Metrics on mini set
if 'gold' in my_reviews.columns:
    y_true = my_reviews['gold'].to_numpy()
    acc = accuracy_score(y_true, pred_labels)
    f1  = f1_score(y_true, pred_labels)
    prec = precision_score(y_true, pred_labels, zero_division=0)
    rec  = recall_score(y_true, pred_labels,    zero_division=0)
    print(f"\nMini-set metrics @ threshold {threshold:.2f}: "
          f"Accuracy={acc:.2%} | F1={f1:.2%} | Precision={prec:.2%} | Recall={rec:.2%}")
    print("Confusion matrix (rows=true, cols=pred):\n", confusion_matrix(y_true, pred_labels))

### Conclusions on my “real” reviews — Model 1 (TF-IDF + LR)

**Mini-set results @ threshold 0.50**  
- Accuracy: **66.7%**, F1: **66.7%** (Precision **63.6%**, Recall **70.0%**)  
- Confusion matrix (rows=true, cols=pred):  
  `[[TN=7, FP=4], [FN=3, TP=7]]` → **11 negatives**, **10 positives**

**What went well**  
- Clear, strongly worded opinions were classified correctly (e.g., *“delightful surprise”*, *“waste of two hours”*, *“heartfelt and beautifully shot”*).  
- The model handled many negations, getting *“I would not recommend this…”* and *“Not even the soundtrack could save this mess.”* right.

**Where it stumbled (examples)**  
- **Negation/contrast with praise** → *“It’s not bad at all—actually pretty great.”* predicted **negative** (FN).  
- **Subtle/ironic tone** → *“Yeah, totally ‘hilarious’—I didn’t laugh once.”* predicted **positive** (FP).  
- **Mixed/nuanced phrasing** → *“Technically impressive, emotionally empty.”* predicted **positive** (FP).  
- **Borderline probabilities near 0.5** → *“Better than the last one by a mile.”* (0.48) and  
  *“This is the kind of movie that stays with you.”* (0.50) flipped to the wrong side due to threshold.

**Takeaways**  
- On this small, deliberately tricky set, performance drops compared to the main test set (where F1 ≈ 0.90).  
- Most errors are **edge cases**: sarcasm, nuanced praise/critique, and borderline confidences. That’s typical for bag-of-ngrams models.

**Easy improvements to try**  
- **Tune the decision threshold** using the F1-vs-threshold curve (e.g., try 0.45 on this mini set to reduce false negatives like *“not bad … pretty great”*).  
- Ensure tokenization preserves negation cues (keep apostrophes; treat “n’t” → “not”)—already done, but verify in the vectorizer.  
- Add a few **hard examples** (sarcasm/contrast) to the training set or use data augmentation.  

**Bottom line:** The model is strong on clear sentiment but still trips on **sarcasm and nuanced wording**. With threshold tuning and a bit of targeted data, I can likely bump this mini-set F1 noticeably.


<div class="alert alert-success">
  <b>Reviewer’s comment – Iteration 1:</b><br>
  Excellent work testing your model on a hand-crafted mini-dataset! You not only ran metrics (accuracy, F1, precision, recall, confusion matrix), but also analyzed specific false positives/negatives and explained why the model struggled (sarcasm, nuanced tone, borderline probabilities). The structured conclusion (what went well, where it stumbled, takeaways, easy improvements) is clear and professional.
</div>


### Model 3

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import numpy as np

# Existing vectorizer
try:
    tfidf_vectorizer_3
except NameError:
    tfidf_vectorizer_3 = tfidf_3  # alias if you named it tfidf_3 before

# Preprocessed spaCy function
texts = my_reviews['review_norm']
texts_m3 = texts.apply(text_preprocessing_3)

X_m3 = tfidf_vectorizer_3.transform(texts_m3)
my_reviews_pred_prob_m3 = model_3.predict_proba(X_m3)[:, 1]

threshold = 0.5
pred_labels_m3 = (my_reviews_pred_prob_m3 >= threshold).astype(int)

print("Model 3 — per-review predictions (p=probability of positive, pred=label)")
for i, preview in enumerate(my_reviews['review'].str.slice(0, 100)):
    print(f"p={my_reviews_pred_prob_m3[i]:.2f}  pred={pred_labels_m3[i]}  |  {preview}")

if 'gold' in my_reviews.columns:
    y_true = my_reviews['gold'].to_numpy()
    acc = accuracy_score(y_true, pred_labels_m3)
    f1  = f1_score(y_true, pred_labels_m3)
    prec = precision_score(y_true, pred_labels_m3, zero_division=0)
    rec  = recall_score(y_true, pred_labels_m3,    zero_division=0)
    print(f"\nModel 3 mini-set @ {threshold:.2f}: "
          f"Accuracy={acc:.2%} | F1={f1:.2%} | Precision={prec:.2%} | Recall={rec:.2%}")
    print("Confusion matrix (rows=true, cols=pred):\n", confusion_matrix(y_true, pred_labels_m3))


### Conclusions on my “real” reviews — Model 3 (spaCy + TF-IDF + LR) @ threshold = 0.50

**Mini-set results:**  
- **Accuracy:** 90.48%  **F1:** 90.00%  (**Precision:** 90.00%, **Recall:** 90.00%)  
- Confusion matrix (rows=true, cols=pred): `[[TN=10, FP=1], [FN=1, TP=9]]` → **11 negatives**, **10 positives**

**Highlights**
- Strong, balanced performance: only **one false positive** and **one false negative**.
- Borderline but correct: *“Technically impressive, emotionally empty.”* → **neg** with **p=0.49** (right on the edge).
- Handles sarcasm better than Model 1: *“Yeah, totally 'hilarious'—I didn’t laugh once.”* → **neg** (p=0.41).
- Confident on clear positives: *“Not only entertaining but also thoughtful.”* (p=0.75),  
  *“This is the kind of movie that stays with you.”* (p=0.60).

**Remaining miss**
- *“I kept checking the time—never a good sign.”* predicted **positive** (p=0.82) → a **false positive**; the phrasing is subtle/idiomatic.

**Takeaways**
- spaCy lemmatization + TF-IDF shows **better robustness** to nuance than Model 1 on this mini set.
- With only one FP/FN, the current **0.50 threshold** is already well-calibrated. If I wanted to be stricter on negatives, I could raise the threshold slightly to reduce FPs (at the cost of some recall).

**Bottom line:** Model 3 generalizes well to my out-of-sample reviews and comfortably meets the project target on this mini evaluation.


<div class="alert alert-success">
  <b>Reviewer’s comment – Iteration 1:</b><br>
  Fantastic job running your spaCy + TF-IDF + Logistic Regression model on the custom mini dataset. The metrics (Accuracy 90%, F1 90%) are very strong, and your structured commentary (highlights, remaining miss, takeaways, bottom line) shows deep understanding. I especially like how you tied specific predictions (sarcasm, borderline cases, idioms) back to why the model succeeded or failed. This is exemplary error analysis.
</div>

### Model 4

In [ ]:

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import numpy as np


tfidf_vectorizer_4 = tfidf_vectorizer_3

texts = my_reviews['review_norm']
texts_m4 = texts.apply(text_preprocessing_3)

X_m4 = tfidf_vectorizer_4.transform(texts_m4)
my_reviews_pred_prob_m4 = model_4.predict_proba(X_m4)[:, 1]

threshold = 0.5
pred_labels_m4 = (my_reviews_pred_prob_m4 >= threshold).astype(int)

print("Model 4 — per-review predictions (p=probability of positive, pred=label)")
for i, preview in enumerate(my_reviews['review'].str.slice(0, 100)):
    print(f"p={my_reviews_pred_prob_m4[i]:.2f}  pred={pred_labels_m4[i]}  |  {preview}")

if 'gold' in my_reviews.columns:
    y_true = my_reviews['gold'].to_numpy()
    acc = accuracy_score(y_true, pred_labels_m4)
    f1  = f1_score(y_true, pred_labels_m4)
    prec = precision_score(y_true, pred_labels_m4, zero_division=0)
    rec  = recall_score(y_true, pred_labels_m4,    zero_division=0)
    print(f"\nModel 4 mini-set @ {threshold:.2f}: "
          f"Accuracy={acc:.2%} | F1={f1:.2%} | Precision={prec:.2%} | Recall={rec:.2%}")
    print("Confusion matrix (rows=true, cols=pred):\n", confusion_matrix(y_true, pred_labels_m4))


### Conclusions on my “real” reviews — Model 4 (spaCy + TF-IDF + LightGBM) @ threshold = 0.50

**Mini-set results:**  
- **Accuracy:** 71.43%  **F1:** 76.92% (**Precision:** 62.50%, **Recall:** 100.00%)  
- Confusion matrix (rows=true, cols=pred): `[[TN=5, FP=6], [FN=0, TP=10]]` → **11 negatives**, **10 positives**

**What this says about the model**
- It’s tuned toward **catching positives** (Recall = **100%**); it didn’t miss any positive review on this mini set.  
- The trade-off is **low precision** (many **false positives**). Six negatives were labeled positive.

**Typical errors (false positives)**
- **Subtle/ironic negatives** marked positive:  
  *“Yeah, totally ‘hilarious’—I didn’t laugh once.”* (**p=0.77**)  
  *“I kept checking the time—never a good sign.”* (**p=0.86**)  
  *“Technically impressive, emotionally empty.”* (**p=0.72**)  
  *“I regret renting this.”* (**p=0.66**)  
- Likely cause: boosting on sparse n-grams can over-weight isolated positive cue words (*impressive*, *hilarious*, *kind of movie*, etc.) and miss the **overall negative composition** or sarcasm.

**How to improve**
- **Raise the decision threshold** (e.g., 0.60–0.65) to trade a bit of recall for better precision on negatives. Pick it from the **PR/F1 curves**.  
- Add more **sarcasm/contrast** examples to training (or reweight such cases).  
- Try **regularizing LightGBM** more (`num_leaves↓`, `min_child_samples↑`, `feature_fraction/colsample_bytree↓`).  
- Consider **TF-IDF → TruncatedSVD (300–500 dims) → LightGBM** to reduce sparsity and soften spurious n-grams.  
- In this project, **Model 3 (LR)** handled nuance better on the same mini set, so it remains the safer default.

**Bottom line:** Model 4 is **recall-heavy** and meets the goal on the main test set, but on these hand-written reviews it flags too many negatives as positive. A higher threshold and some regularization should close the gap.


<div class="alert alert-success">
  <b>Reviewer’s comment – Iteration 1:</b><br>
  Excellent follow-up evaluation of your LightGBM model on the custom mini dataset. You clearly documented results (Acc ≈ 71%, F1 ≈ 77%, Recall = 100%) and explained the trade-off between recall and precision. The error analysis with concrete false positive examples (sarcasm, subtle negatives) is very strong and shows a nuanced understanding of how boosting can overweight isolated cue words in sparse text.
</div>


### Model 9

In [ ]:

import numpy as np
from sklearn.metrics import (
    f1_score, average_precision_score, roc_auc_score,
    precision_recall_curve, accuracy_score, precision_score, recall_score, confusion_matrix
)
from sklearn.model_selection import StratifiedShuffleSplit

# 1) Validation split from TRAIN (to tune threshold without touching test)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
(tr_idx, val_idx) = next(sss.split(train_features_9, target_train_bert))
feat_tr9, target_tr9   = train_features_9[tr_idx], target_train_bert[tr_idx]
feat_val9, target_val9 = train_features_9[val_idx], target_train_bert[val_idx]

# model_9 refit on the training portion
model_9.fit(feat_tr9, target_tr9)

# 2) Threshold for the NEGATIVE class
#    negatives (label==0) as the "positive" class for metric purposes
val_p_pos = model_9.predict_proba(feat_val9)[:, 1]     # P(positive)
val_p_neg = 1.0 - val_p_pos                         # P(negative)

ts = np.linspace(0, 1, 401)
f1s = [f1_score((target_val9==0).astype(int), (val_p_neg >= t).astype(int)) for t in ts]
t_star = ts[int(np.argmax(f1s))]
print(f"Best threshold for NEGATIVE on validation: {t_star:.2f} (F1={max(f1s):.3f})")

# 3) Final evaluation on TEST at tuned threshold (NEGATIVE focus)
test_p_pos = model_9.predict_proba(test_features_9)[:, 1]
test_p_neg = 1.0 - test_p_pos

target_true_neg = (target_test_bert==0).astype(int)
target_pred_neg = (test_p_neg >= t_star).astype(int)

acc  = accuracy_score(target_true_neg, target_pred_neg)
f1   = f1_score(target_true_neg, target_pred_neg)
prec = precision_score(target_true_neg, target_pred_neg, zero_division=0)
rec  = recall_score(target_true_neg, target_pred_neg,    zero_division=0)
aps  = average_precision_score(target_true_neg, test_p_neg)  # PR AUC for negative detection
roc  = roc_auc_score(target_true_neg, test_p_neg)
cm   = confusion_matrix(target_true_neg, target_pred_neg)

print(f"\nModel 9 — NEGATIVE detection @ threshold {t_star:.2f}")
print(f"Accuracy={acc:.3f} | F1={f1:.3f} | Precision={prec:.3f} | Recall={rec:.3f}")
print(f"APS (PR AUC)={aps:.3f} | ROC AUC={roc:.3f}")
print("Confusion matrix (rows=true_neg, cols=pred_neg):\n", cm)

# 4) POSITIVE-class metrics for completeness
target_true_pos = (target_test_bert==1).astype(int)
target_pred_pos = (test_p_pos >= (1 - t_star)).astype(int)  # complementary threshold
print("\n(Also for POSITIVE class)")
print("F1_pos=", f1_score(target_true_pos, target_pred_pos))
print("APS_pos=", average_precision_score(target_true_pos, test_p_pos), " ROC_pos=", roc_auc_score(target_true_pos, test_p_pos))




### Model 9 (BERT embeddings + LR) — Focusing on **both** classes

**Validation tuning:** I optimized the decision threshold to better detect **negative** reviews, which gave  
**t\* = 0.31** on the *negative* score (i.e., predict negative if *p(neg) ≥ 0.31*).

**Test results at t\* = 0.31**

- **Negative class (our tuned target)**
  - **F1_neg:** **0.798**  | **Precision_neg:** 0.744  | **Recall_neg:** **0.861**
  - **APS_neg (PR AUC):** 0.827  | **ROC AUC_neg:** 0.817

- **Positive class** *(complementary threshold; p(pos) ≥ 0.69)*  
  - **F1_pos:** **0.719**  | **APS_pos:** 0.811  | **ROC AUC_pos:** 0.817

- **Overall**
  - **Accuracy:** 0.765  
  - **Macro-F1 (avg of pos/neg F1):** ≈ **0.759**  
  - **Confusion matrix** *(rows = true_neg, cols = pred_neg)*:
    ```
    [[60 32]
     [15 93]]
    ```
    → 92 positive reviews (row 0), 108 negative reviews (row 1).  
      Correct: 60 true positives (for the positive class) + 93 true negatives (for the negative class).  
      Errors: 32 positives flagged as negative (FP_neg) and 15 negatives missed (FN_neg).

**Interpretation (both classes)**
- With t\* = 0.31 the model is **recall-oriented for negatives** (it catches most negative reviews), which is often desirable if the goal is to surface harmful content or dissatisfied users.  
- The **positive class** remains solid (F1 ≈ 0.72), so performance is reasonably balanced across both sides.
- The ranking quality is consistent for both classes (**APS/ROC ≈ 0.81–0.83**), meaning scores are well ordered even before thresholding.

**If I want a more symmetric balance**
- I can re-tune the threshold to **maximize Macro-F1** (averaging F1 for positive and negative) on a validation split.  
- Alternatively, use **dual thresholds** for triage:
  - `p(pos) ≥ 0.7` → confidently **positive**  
  - `p(pos) ≤ 0.3` → confidently **negative**  
  - `0.3 < p(pos) < 0.7` → **manual review** / second pass  
  This reduces mistakes at the cost of leaving some reviews unclassified automatically.

**Conclusion**  
Model 9 performs well for **both positive and negative** detection. At the negative-optimized threshold it delivers high **F1_neg (0.80)** while keeping **F1_pos (~0.72)** respectable. Depending on project priorities (catch more negatives vs. fewer false alarms), I can either keep t\* = 0.31, re-tune for Macro-F1, or adopt dual thresholds for production.

## Conclusions

**Goal:** Build a model to automatically detect negative IMDB reviews and reach **F1 ≥ 0.85** on the held-out test set.

### What I built
- Loaded and cleaned the IMDB dataset; applied **light normalization** (HTML → space, keep apostrophes, lowercase).
- Verified **no class imbalance** (≈ 50/50 pos/neg across train/test).
- Created multiple text pipelines:
  - **Model 0:** Dummy (constant/most frequent) — sanity baseline.
  - **Model 1:** `review_norm` → **TF-IDF (1–2 grams)** → **LogisticRegression** (+ NLTK prep variant).
  - **Model 3:** **spaCy lemmatization** → TF-IDF (1–2) → LogisticRegression.
  - **Model 4:** **spaCy lemmas** → TF-IDF (1–2) → **LightGBM**.
  - **Model 9:** **BERT [CLS] embeddings** → LogisticRegression (threshold tuned on validation).

### Core results (held-out test)
- **Model 0 (Dummy):** Accuracy ≈ 0.50, **F1 = 0.00** — as expected (no skill).
- **Model 1 (TF-IDF + LR, NLTK):** **Accuracy = 0.90, F1 = 0.90, APS = 0.96, ROC AUC = 0.97.** ✅ *Meets target.*
- **Model 3 (spaCy + TF-IDF + LR):** **Accuracy = 0.89, F1 = 0.89, APS = 0.96, ROC AUC = 0.96.** ✅ *Meets target.*
- **Model 4 (spaCy + TF-IDF + LightGBM):** **Accuracy = 0.89, F1 = 0.89, APS = 0.95, ROC AUC = 0.96.** ✅ *Meets target.*
- **Model 9 (BERT embeddings + LR, tuned for negatives):**  
  Test at tuned threshold **t\* = 0.31** on *p(negative)*  
  – **F1_neg = 0.80, F1_pos ≈ 0.72, Accuracy = 0.77, APS ≈ 0.82, ROC AUC ≈ 0.82.**  
  *Good ranking, but below the TF-IDF+LR baselines in F1 on CPU-sized runs.*

**Bottom line:** Both **Model 1** and **Model 3** comfortably surpass the **0.85 F1** requirement on the official test set.  
Model 4 is close but not clearly better than LR on sparse TF-IDF.  
Model 9 is promising but underperforms the linear baselines at this scale/hardware.

### “Real” out-of-sample reviews I wrote (mini-set check)
- I authored a small set of new reviews with **known labels** to test generalization.
- **Model 1** at threshold 0.45: **F1 = 0.75** (Recall strong, a few false positives on sarcasm/nuance).  
- **Model 3** at threshold 0.50: **F1 = 0.90** (only one FP and one FN; best robustness on this mini set).  
- **Model 4** at 0.50: **F1 = 0.77** with **Recall = 1.00** (caught all positives but flagged too many negatives as positive).

**Takeaway from the mini set:** **Model 3 (spaCy + TF-IDF + LR)** handled sarcasm/contrast slightly better and was most balanced on novel text.

### Final choice & operating threshold
- **Selected model for delivery:** **Model 3 — spaCy + TF-IDF (1–2) + LogisticRegression**  
  - **Rationale:** Highest reliability on my out-of-sample reviews, competitive test F1, fast, simple, and easy to maintain.
- **Threshold policy:**  
  - If the business goal is to **catch more negatives**, use **t ≈ 0.45** on *p(positive)* (or equivalently lower the positive threshold) to improve recall.  
  - If you want **balanced precision/recall** and clean confusion matrices, **t = 0.50** is already strong.

### Error patterns & what I learned
- **Sarcasm/irony, nuanced contrast, and idioms** (e.g., “emotionally empty”, “never a good sign”) are the main sources of mistakes.
- Keeping **apostrophes** and **bigrams** is crucial for **negation** (e.g., “not good”, “didn’t like”).

### Next steps (if I iterate further)
1. **Threshold tuning** on a validation split targeted to the deployment objective (macro-F1 vs. negative-recall).  
2. **Light calibration** or **Platt/Isotonic** if well-calibrated probabilities matter.  
3. Try **TF-IDF → TruncatedSVD (300–500 dims)** → LR/LightGBM to reduce sparsity and spurious n-grams.  
4. If GPU is available, revisit **DistilBERT/BERT** end-to-end fine-tuning (small epochs, gradual scale-up).  
5. **Data augmentation** with more sarcastic/contrastive examples to reduce those specific errors.

---

**Conclusion:** The project objective is achieved.  
I recommend shipping **Model 3 (spaCy + TF-IDF + LogisticRegression)** with a documented threshold (0.50 by default, 0.45 if negative-recall is prioritized). It’s accurate (F1 ≈ **0.89–0.90** on test), fast, and interpretable, and it held up best on fresh reviews I wrote.


# Checklist

- [x]  Notebook was opened
- [x]  The text data is loaded and pre-processed for vectorization
- [x]  The text data is transformed to vectors
- [x]  Models are trained and tested
- [x]  The metric's threshold is reached
- [x]  All the code cells are arranged in the order of their execution
- [x]  All the code cells can be executed without errors
- [x]  There are conclusions